In [5]:
!pip install sentencepiece protobuf

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install accelerate bitsandbytes peft trl datasets psutil

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import AutoModelForCausalLM, LlamaTokenizer
import torch

# আমরা 1-bit LLM এর কমিউনিটি ভার্সন ব্যবহার করছি
model_name = "1bitLLM/bitnet_b1_58-3B"

print(f"Loading {model_name}...")

try:
    # ১. টোকেনাইজার লোড (LlamaTokenizer ব্যবহার করা হচ্ছে)
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    
    # ২. মডেল লোড
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
    
    print("Model Loaded Successfully!")

    # ৩. টেস্ট রান
    prompt = "Explain Artificial Intelligence in one sentence."
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # জেনারেশন
    outputs = model.generate(**inputs, max_new_tokens=50)
    print("-" * 50)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("-" * 50)

except Exception as e:
    print(f"Error: {e}")

Loading 1bitLLM/bitnet_b1_58-3B...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

e:\SmartSearchSystem\SmartSearchSystem\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\RUET\.cache\huggingface\hub\models--1bitLLM--bitnet_b1_58-3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.31G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Model Loaded Successfully!
--------------------------------------------------
Explain Artificial Intelligence in one sentence.
--------------------------------------------------


In [5]:
# টেক্সট কমপ্লিশন টেস্ট
input_text = "The capital of Bangladesh is"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

print("Generating...")
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.7)

print("-" * 50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print("-" * 50)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [6]:
# ১. প্যাডিং টোকেন ফিক্স (LlamaTokenizer-এর জন্য এটি জরুরি)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ২. ডিভাইস নির্ধারণ (সরাসরি 'cuda' ব্যবহার করা নিরাপদ)
device = "cuda" if torch.cuda.is_available() else "cpu"

# ৩. ইনপুট টেক্সট তৈরি
input_text = "The capital of Bangladesh is"

# ৪. টোকেনাইজ করা (সরাসরি জিপিউতে পাঠানো হচ্ছে না)
inputs = tokenizer(input_text, return_tensors="pt")

# ৫. ইনপুটগুলোকে ম্যানুয়ালি জিপিউতে পাঠানো (এটি RuntimeError আটকাবে)
inputs = {key: value.to(device) for key, value in inputs.items()}

print("Generating...")

# ৬. জেনারেশন (প্যারামিটারগুলো একটু টিউন করা হয়েছে ভালো ফলাফলের জন্য)
with torch.no_grad():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=50, 
        do_sample=True, 
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

print("-" * 50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print("-" * 50)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [1]:
from transformers import AutoModelForCausalLM, LlamaTokenizer
import torch

# মডেল নাম
model_name = "1bitLLM/bitnet_b1_58-3B"

print("Loading model in SAFE MODE (Float32)...")

try:
    # ১. টোকেনাইজার
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # ২. মডেল লোড (এখানেই পরিবর্তন: float32 ব্যবহার করা হচ্ছে)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32, # <--- পরিবর্তন: ১৬ এর বদলে ৩২ (সেফ মোড)
        device_map="auto",
        trust_remote_code=True
    )
    
    # সেফটি বাফার (আগের ফিক্সটি রাখলাম)
    model.resize_token_embeddings(len(tokenizer) + 64)
    
    print("Model Loaded Successfully in Float32!")

    # ৩. ইনপুট
    input_text = "Artificial Intelligence is"
    inputs = tokenizer(input_text, return_tensors="pt")
    
    # ডিভাইসে পাঠানো
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    print("Generating...")
    
    # ৪. জেনারেশন
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=50, 
            do_sample=True, 
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    print("-" * 50)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("-" * 50)

except Exception as e:
    print(f"Error: {e}")

Loading model in SAFE MODE (Float32)...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BitnetTokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Model Loaded Successfully in Float32!
Generating...
--------------------------------------------------
Artificial Intelligence is adjacent private traddan snow cleared BreisBre objective Old Bud Old private absood variabel bre suppis sob horsesis abs tasteool Breisdan driveribus Breisibus flagsagemiji joy Breisobb pole Breis flags Bre suppthen recover
--------------------------------------------------


In [3]:
import torch

if torch.cuda.is_available():
    # মোট মেমোরি
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    # বর্তমানে ব্যবহার হচ্ছে (Allocated)
    allocated_mem = torch.cuda.memory_allocated(0) / 1024**3
    
    # রিজার্ভ করা আছে (Reserved - ক্যাশ সহ)
    reserved_mem = torch.cuda.memory_reserved(0) / 1024**3
    
    # খালি আছে
    free_mem = total_mem - reserved_mem
    
    print("-" * 30)
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print("-" * 30)
    print(f"Total VRAM:     {total_mem:.2f} GB")
    print(f"Used VRAM:      {reserved_mem:.2f} GB")
    print(f"Free VRAM:      {free_mem:.2f} GB")
    print("-" * 30)
    
    if free_mem < 2:
        print("⚠️ Warning: Low VRAM! Restart kernel to free up space.")
    else:
        print("✅ You have enough VRAM to load more models.")
else:
    print("❌ No GPU detected!")

------------------------------
GPU Name: NVIDIA RTX A4500
------------------------------
Total VRAM:     19.99 GB
Used VRAM:      13.45 GB
Free VRAM:      6.54 GB
------------------------------
✅ You have enough VRAM to load more models.


In [4]:
!pip install llama-cpp-python huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/50.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/50.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/50.7 MB ? eta -:--:--
     ---------------------------------------- 0.3/50.7 MB ? eta -:--:--
     ---------------------------------------- 0.3/50.7 MB ? eta -:--:--
     ---------------------------------------- 0.3/50.7 MB ? eta -:--:--
     --------------------------------------- 0.5/50.7 MB 465.6 kB/s eta 0:01:48
     --------------------------------------- 0.5/50.7 MB 465.6 kB/s eta 0:01:48
     --------------------------------------- 0.5/50.7 MB 465.6 kB/s eta 0:01:48
     --------------------------------------- 0.5/50.7 MB 465.6 kB/s eta 0:01:48
      -------------------------------------- 0.8/50.7 MB 364.6 kB/s eta 0:02:17
      -------------------------------------- 0.8/50.7 MB 364.6 kB/s eta 0:02:17
      ---------

  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.11.6 using CMake 4.2.1 (wheel)
      *** Configuring CMake...
      2026-01-22 22:09:59,060 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\RUET\AppData\Local\Temp\tmppsn2xnag\build\CMakeInit.txt
      -- Building for: NMake Makefiles
      CMake Error at CMakeLists.txt:3 (project):
        Running
      
         'nmake' '-?'
      
        failed with:
      
         no such file or directory
      
      
      CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
      CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
      -- Configuring incomplete, errors occurred!
      
      *** CMake configuration failed
      [end of output]
  
  note: This error ori

In [1]:
!pip install llama-cpp-python --prefer-binary --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com, https://abetlen.github.io/llama-cpp-python/whl/cu124
     ---------------------------------------- 0.0/443.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/443.4 MB ? eta -:--:--
     ---------------------------------------- 0.3/443.4 MB ? eta -:--:--
     ---------------------------------------- 0.3/443.4 MB ? eta -:--:--
     -------------------------------------- 0.5/443.4 MB 578.7 kB/s eta 0:12:46
     -------------------------------------- 0.5/443.4 MB 578.7 kB/s eta 0:12:46
     -------------------------------------- 0.5/443.4 MB 578.7 kB/s eta 0:12:46
     -------------------------------------- 0.5/443.4 MB 578.7 kB/s eta 0:12:46
     -------------------------------------- 0.5/443.4 MB 578.7 kB/s eta 0:12:46
     -------------------------------------- 0.5/443.4 MB 578.7 kB/s eta 0:12:46
     -------------------------------------- 0.5/443.4 MB 578.7 kB/s eta 0:12:46
     ----------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import os

# ১. TinyLlama মডেল (এটি খুব স্টেবল এবং ছোট)
model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
model_file = "tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"

print(f"Downloading Diagnostic Model: {model_name}...")

try:
    model_path = hf_hub_download(repo_id=model_name, filename=model_file)
    print(f"Path: {model_path}")

    # ২. লোড টেস্ট
    print("Testing GPU Loading...")
    llm = Llama(
        model_path=model_path,
        n_gpu_layers=-1,
        verbose=False
    )
    
    # ৩. জেনারেশন টেস্ট
    print("Generating...")
    output = llm("Hello! Who are you?", max_tokens=50)
    print("-" * 30)
    print("Success! Output:", output['choices'][0]['text'])
    print("-" * 30)
    print("✅ System is OK. The previous BitNet file was likely corrupted.")

except Exception as e:
    print(f"❌ System Error: {e}")
    print("Your llama-cpp-python installation might be broken.")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf:   0%|          | 0.00/669M [00:00<?, ?B/s]

e:\SmartSearchSystem\SmartSearchSystem\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\RUET\.cache\huggingface\hub\models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Path: C:\Users\RUET\.cache\huggingface\hub\models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF\snapshots\52e7645ba7c309695bec7ac98f4f005b139cf465\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf
Testing GPU Loading...


llama_new_context_with_model: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized


Generating...
------------------------------
Success! Output:  You're a person! I'm a computer. What's your name? I don't have a name. You're a computer! What's your name? I don't have a name. You're a
------------------------------
✅ System is OK. The previous BitNet file was likely corrupted.


In [5]:
import torch
import os

print(f"PyTorch is using CUDA Version: {torch.version.cuda}")
print("-" * 50)

# ড্রাইভারের ইনফরমেশন দেখার জন্য
# এটি দেখাবে আপনার জিপিউ ড্রাইভার সর্বোচ্চ কত ভার্সন সাপোর্ট করে
print("NVIDIA Driver Info:")
!nvidia-smi

PyTorch is using CUDA Version: 12.4
--------------------------------------------------
NVIDIA Driver Info:
Thu Jan 22 22:14:43 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.15                 Driver Version: 581.15         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A4500             WDDM  |   00000000:02:00.0  On |                  Off |
| 30%   29C    P8             12W /  200W |   14727MiB /  20470MiB |      2%      Default |
|                                

In [ ]:
import asyncio
import json
import logging
import time
from typing import Dict, List, Optional, Union
from dataclasses import dataclass
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

@dataclass
class BitNETRequest:
    """Structured request for BitNET service"""
    id: str
    prompt: str
    max_tokens: int = 256
    temperature: float = 0.7
    top_p: float = 0.9
    stream: bool = False
    metadata: Optional[Dict] = None

@dataclass
class BitNETResponse:
    """Structured response from BitNET service"""
    id: str
    response: str
    generation_time: float
    tokens_generated: int
    tokens_per_second: float
    success: bool
    error_message: Optional[str] = None
    metadata: Optional[Dict] = None

class ProductionBitNETService:
    """Production-ready BitNET service with enterprise features"""
    
    def __init__(
        self,
        model_name: str = "microsoft/bitnet-b1.58-2B-4T",
        max_concurrent_requests: int = 10,
        request_timeout: float = 30.0,
        enable_caching: bool = True
    ):
        self.model_name = model_name
        self.max_concurrent_requests = max_concurrent_requests
        self.request_timeout = request_timeout
        self.enable_caching = enable_caching
        
        # Service state
        self.model = None
        self.tokenizer = None
        self.request_semaphore = asyncio.Semaphore(max_concurrent_requests)
        self.request_cache = {}
        self.metrics = {
            "total_requests": 0,
            "successful_requests": 0,
            "failed_requests": 0,
            "total_generation_time": 0.0,
            "total_tokens_generated": 0
        }
        
        # Setup logging
        self.logger = self._setup_logging()
        
    def _setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - BitNET-Production - %(levelname)s - %(message)s'
        )
        return logging.getLogger("BitNET-Production")
    
    async def initialize(self):
        try:
            self.logger.info(f"Initializing BitNET service with model: {self.model_name}")
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )
            self.model.eval()
            self.logger.info("BitNET service initialized successfully")
        except Exception as e:
            self.logger.error(f"Failed to initialize BitNET service: {str(e)}")
            raise
    
    def _generate_cache_key(self, request: BitNETRequest) -> str:
        key_data = {
            "prompt": request.prompt,
            "max_tokens": request.max_tokens,
            "temperature": request.temperature,
            "top_p": request.top_p
        }
        return str(hash(json.dumps(key_data, sort_keys=True)))
    
    async def _generate_internal(self, request: BitNETRequest) -> BitNETResponse:
        start_time = time.time()
        try:
            if self.enable_caching:
                cache_key = self._generate_cache_key(request)
                if cache_key in self.request_cache:
                    cached_response = self.request_cache[cache_key]
                    return BitNETResponse(
                        id=request.id,
                        response=cached_response["response"],
                        generation_time=cached_response["generation_time"],
                        tokens_generated=cached_response["tokens_generated"],
                        tokens_per_second=cached_response["tokens_per_second"],
                        success=True,
                        metadata={"cache_hit": True}
                    )
            
            messages = [{"role": "user", "content": request.prompt}]
            formatted_prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(formatted_prompt, return_tensors="pt", truncation=True, max_length=2048).to(self.model.device)
            
            generation_start = time.time()
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=request.max_tokens,
                    temperature=request.temperature,
                    top_p=request.top_p,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    early_stopping=True
                )
            
            generation_time = time.time() - generation_start
            response_text = self.tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()
            tokens_generated = outputs.shape[1] - inputs['input_ids'].shape[1]
            tokens_per_second = tokens_generated / generation_time if generation_time > 0 else 0
            
            response = BitNETResponse(
                id=request.id,
                response=response_text,
                generation_time=generation_time,
                tokens_generated=tokens_generated,
                tokens_per_second=tokens_per_second,
                success=True,
                metadata={"model": self.model_name, "cache_hit": False}
            )
            
            if self.enable_caching:
                cache_key = self._generate_cache_key(request)
                self.request_cache[cache_key] = {
                    "response": response_text,
                    "generation_time": generation_time,
                    "tokens_generated": tokens_generated,
                    "tokens_per_second": tokens_per_second
                }
            
            return response
            
        except Exception as e:
            self.logger.error(f"Generation failed: {str(e)}")
            return BitNETResponse(id=request.id, response="", generation_time=time.time()-start_time, tokens_generated=0, tokens_per_second=0, success=False, error_message=str(e))

    async def generate(self, request: BitNETRequest) -> BitNETResponse:
        self.metrics["total_requests"] += 1
        async with self.request_semaphore:
            try:
                response = await asyncio.wait_for(self._generate_internal(request), timeout=self.request_timeout)
                return response
            except asyncio.TimeoutError:
                return BitNETResponse(id=request.id, response="", generation_time=self.request_timeout, tokens_generated=0, tokens_per_second=0, success=False, error_message="Request timed out")

# --- Run the Service ---
async def run_production_demo():
    service = ProductionBitNETService()
    await service.initialize()
    
    req = BitNETRequest(id="1", prompt="What makes BitNET efficient?", max_tokens=100)
    res = await service.generate(req)
    
    print(f"Response: {res.response}")
    print(f"Speed: {res.tokens_per_second:.2f} t/s")

# Uncomment to run in notebook:
# await run_production_demo()

In [ ]:
import hashlib
import threading
import re

@dataclass
class SecurityConfig:
    max_input_length: int = 4096
    max_output_tokens: int = 1024
    rate_limit_requests_per_minute: int = 60
    enable_content_filtering: bool = True
    log_requests: bool = True
    sanitize_inputs: bool = True

class SecureBitNETService:
    def __init__(self, model_name="microsoft/bitnet-b1.58-2B-4T", security_config=None):
        self.model_name = model_name
        self.security_config = security_config or SecurityConfig()
        self.request_history = {}
        self.security_lock = threading.Lock()
        
        # Load model (Assuming standard loading for brevity)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)
    
    def _rate_limit_check(self, client_id: str) -> bool:
        hashed_id = hashlib.sha256(client_id.encode()).hexdigest()[:16]
        current_time = time.time()
        with self.security_lock:
            if hashed_id not in self.request_history:
                self.request_history[hashed_id] = []
            self.request_history[hashed_id] = [t for t in self.request_history[hashed_id] if current_time - t < 60]
            if len(self.request_history[hashed_id]) >= self.security_config.rate_limit_requests_per_minute:
                return False
            self.request_history[hashed_id].append(current_time)
            return True

    def secure_generate(self, prompt: str, client_id: str, max_tokens=200):
        if not self._rate_limit_check(client_id):
            return {"success": False, "error": "Rate limit exceeded"}
        
        # Basic sanitization
        if self.security_config.sanitize_inputs:
            prompt = re.sub(r"<script[^>]*>.*?</script>", "", prompt, flags=re.IGNORECASE | re.DOTALL)
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_new_tokens=max_tokens)
        
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return {"success": True, "response": response}

# Usage
secure_service = SecureBitNETService()
print(secure_service.secure_generate("Hello world", client_id="user123"))

In [ ]:
import psutil
import statistics

@dataclass
class PerformanceSnapshot:
    timestamp: float
    memory_usage_mb: float
    cpu_usage_percent: float
    gpu_memory_mb: Optional[float]
    tokens_per_second: float

class BitNETMonitoringService:
    def __init__(self):
        self.performance_history = []
    
    def capture_snapshot(self, tokens_generated, generation_time):
        memory_usage = psutil.Process().memory_info().rss / 1024 / 1024
        cpu_usage = psutil.cpu_percent()
        gpu_memory = torch.cuda.memory_allocated() / 1024 / 1024 if torch.cuda.is_available() else 0
        tps = tokens_generated / generation_time if generation_time > 0 else 0
        
        snapshot = PerformanceSnapshot(time.time(), memory_usage, cpu_usage, gpu_memory, tps)
        self.performance_history.append(snapshot)
        return snapshot

    def get_summary(self):
        if not self.performance_history: return "No data"
        avg_tps = statistics.mean([s.tokens_per_second for s in self.performance_history])
        max_mem = max([s.memory_usage_mb for s in self.performance_history])
        return {"avg_tps": avg_tps, "peak_memory_mb": max_mem}

# Usage
monitor = BitNETMonitoringService()
# Simulate a run
monitor.capture_snapshot(tokens_generated=100, generation_time=5.0)
print("Monitoring Summary:", monitor.get_summary())